In [13]:
import time
import copy

In [14]:
class parse:
    def __init__(self, raw, offset = 0):
        self.parse(raw, offset)
        self.stack = []
    def parse(self, raw, offset):
        pass
    def append(self, parent):
        pass
    def timeshift(self, offset):
        pass
        
def CInt(raw, offset):
    return offset+4, int.from_bytes(raw[offset:offset+4], "little")
    
def CLong( raw, offset):
    return offset+8, int.from_bytes(raw[offset:offset+8], "little")

def CString( raw, offset):
    result = raw[offset:].decode("ASCII")
    return offset+len(result), result
    
class Alloc( parse ):
    def parse(self, raw, offset):
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.addr = CLong(raw, offset)
        offset, self.size = CLong(raw, offset)
    def append(self, parent):
        parent["memory"][self.addr] = self
        parent["last"] = parent["memory"][self.addr]
        if self.tid in parent["threads"]:
            parent["threads"][self.tid].consumption += self.size
        parent["consumption"] += self.size
    def timeshift(self, offset):
        self.time -= offset
        
class Free( parse ):
    def parse(self, raw, offset):
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.addr = CLong(raw, offset)
        offset, self.size = CLong(raw, offset)
    def append(self, parent):
        if not self.addr in parent["memory"]:
            parent["memory"][self.addr] = self
        oldsize = parent["memory"][self.addr].size
        parent["memory"][self.addr].size = 0
        parent["last"] = parent["memory"][self.addr]
        if self.tid in parent["threads"]:
            parent["threads"][self.tid].consumption -= oldsize  
        parent["consumption"] -= oldsize
    def timeshift(self, offset):
        self.time -= offset
        
class Thread( parse ):
    def parse(self, raw, offset):
        result = {}
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.child_tid = CLong(raw, offset)
        self.consumption = 0
    def append(self, parent):
        print(f"new thread {self.child_tid}")
        parent["threads"][self.child_tid] = self
        parent["last"] = parent["threads"][self.child_tid]
    def timeshift(self, offset):
        self.time -= offset
    
class Stack( parse ):
    def parse(self, raw, offset):
        result = {}
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.length = CLong(raw, offset)   
    def append(self, parent):
        pass
    def timeshift(self, offset):
        self.time -= offset
    
class Frame( parse ):
    def parse(self, raw, offset):
        offset, self.symbol = CString(raw, offset)
    def append(self, parent):
        parent["last"].stack.append(self.symbol) 
    
actions = { "A": { "type": Alloc },
            "F": { "type": Free },
            "T": { "type": Thread },
            "S": { "type": Stack },
            "s": { "type": Frame }
          }


def evaluate(action, length, data, offset_t):
    item = actions[action]["type"](data)
    item.timeshift(offset_t)
    return item
        
    
def findAndReadLineHeader(log):
    # line must begin with hashtag-action-length tuple, first search for a newline beginning with hashtag
    success = False
    action = None
    length = None
    data = None
    while not success:
        l = b'\0'
        while l != b'#' and len(l) > 0:
            l = log.read(1)
        l = log.read(9)
        action = l[0]
        length = int.from_bytes(l[1:], "little") #8

        # remove newline char
        if length <= 0 or length > 200:
            continue
        data = log.read(length-1)
        newline = log.read(1)
        success = (newline == b"\n")
        if not success:
            print(data, newline)
    return chr(action), length-1, data

def readLine(log, offset_t):
    action, length, data = findAndReadLineHeader(log)
    #print(action, length, data)
    
    item = evaluate(action, length, data, offset_t)
    return item


In [15]:
log = open("mallog.log", "rb")
hist = [{"memory": {}, "threads": {}, "last": None, "consumption": 0}]

In [ ]:
item = readLine(log, 0)
offset_t = item.time
item.time = 0

item.append(hist[-1])

nextts = 1

while True:
    try:
        item = readLine(log, offset_t)
        fails = 100
    except:
        fails -= 1
        if fails == 0:
            break
        continue
    
    if hist[-1]["last"].time >= nextts:
        hist.append(copy.deepcopy(hist[-1]))
        #print()
        print(f"time: {hist[-1]['last'].time} -- {hist[-1]['consumption']/1024/1024} MB")
        nextts = hist[-1]["last"].time + 1
        
    item.append(hist[-1])

print(f"time: {hist[-1]['last'].time} -- {hist[-1]['consumption']/1024/1024} MB")
print("end")


new thread 139709784594176
new thread 139709729736448
new thread 139709721343744
new thread 139709712951040
new thread 139709704558336
new thread 139709696165632
new thread 139709687772928
new thread 139709687772928
new thread 139709687772928
new thread 139709679380224
new thread 139709670987520
new thread 139709662594816
time: 1 -- 6.227544784545898 MB
time: 2 -- 6.226832389831543 MB
time: 3 -- 6.227852821350098 MB
time: 4 -- 6.228331565856934 MB
time: 5 -- 6.227852821350098 MB
time: 6 -- 7.62339973449707 MB
time: 7 -- 6.227901458740234 MB
time: 8 -- 6.227852821350098 MB
time: 9 -- 6.228206634521484 MB
time: 10 -- 6.2276716232299805 MB
time: 11 -- 7.6793107986450195 MB
new thread 139709313447680
time: 12 -- 9.488545417785645 MB
new thread 139709313447680
new thread 139709305054976
new thread 139709296662272
new thread 139709288269568
new thread 139709279876864
new thread 139709279876864
new thread 139709271484160
new thread 139709258880768
new thread 139709254682368
new thread 1397089

In [12]:
threads = hist[-1]["threads"]

[ (threads[x].tid, threads[x].consumption, threads[x].stack) for x in threads ]

[(140693671085632,
  0,
  ['/lib/x86_64-linux-gnu/libstdc++.so.6(_ZNSt6thread15_M_start_threadESt10unique_ptrINS_6_StateESt14def',
   '/usr/local/lib/librealsense2.so.2.49(_ZNSt6threadC2IMN2el4base19AsyncDispatchWorkerEFvvEJPS3_EEEOT_D',
   '/usr/local/lib/librealsense2.so.2.49(_ZN2el4base19AsyncDispatchWorker5startEv+0x5e) [0x7ff5d888acf2]',
   '/usr/local/lib/librealsense2.so.2.49(_ZN2el4base7StorageC1ERKSt10shared_ptrINS_10LogBuilderEEPNS0_7I',
   '/usr/local/lib/librealsense2.so.2.49(+0x15c02f7) [0x7ff5d86aa2f7]',
   '/usr/local/lib/librealsense2.so.2.49(+0x15c04ed) [0x7ff5d86aa4ed]',
   '/lib64/ld-linux-x86-64.so.2(+0x11b8a) [0x7ff5d98d3b8a]',
   '/lib64/ld-linux-x86-64.so.2(+0x11c91) [0x7ff5d98d3c91]',
   '/lib64/ld-linux-x86-64.so.2(+0x113a) [0x7ff5d98c313a]']),
 (140693671085632,
  0,
  ['/lib/x86_64-linux-gnu/libstdc++.so.6(_ZNSt6thread15_M_start_threadESt10unique_ptrINS_6_StateESt14def',
   '/usr/local/lib/libopenvslam.so(_ZN9openvslam6system7startupEb+0x16a) [0x7ff5d6d58e98]